In [ ]:
!pip install tensorflow
!pip install torch torchvision



In [ ]:
import tensorflow as tf
import torch

print("TensorFlow version:", tf.__version__)
print("PyTorch version:", torch.__version__)


In [ ]:
import boto3
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from io import BytesIO
class S3ImageDataset(Dataset):
    def __init__(self, bucket_name, prefix, transform=None):
        """
        S3 버킷에서 이미지를 로드하는 PyTorch 데이터셋 클래스.
        Args:
        - bucket_name: S3 버킷 이름.
        - prefix: S3 폴더 경로 (e.g., 'your-folder/train/').
        - transform: 이미지 전처리를 위한 torchvision.transforms 객체.
        """
        self.s3 = boto3.client('s3')
        self.bucket_name = bucket_name
        self.prefix = prefix
        self.transform = transform
        self.files = self._list_files()
    def _list_files(self):
        """S3 폴더 내 파일 리스트를 가져옵니다."""
        response = self.s3.list_objects_v2(Bucket=self.bucket_name, Prefix=self.prefix)
        return [content['Key'] for content in response.get('Contents', []) if not content['Key'].endswith('/')]
    def __len__(self):
        """데이터셋의 전체 이미지 수."""
        return len(self.files)
    def __getitem__(self, idx):
        """지정된 인덱스의 이미지를 가져옵니다."""
        file_key = self.files[idx]
        response = self.s3.get_object(Bucket=self.bucket_name, Key=file_key)
        image_data = response['Body'].read()
        image = Image.open(BytesIO(image_data)).convert('RGB')  # 이미지 열기
        if self.transform:
            image = self.transform(image)
        # 클래스 레이블 추출 (폴더 이름에서 추출)
        label = 0 if "NG/" in file_key else 1  # NG -> 0, OK -> 1
        return image, label

In [ ]:
# 이미지 전처리 파이프라인 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet에 맞게 크기 조정
    transforms.ToTensor(),         # Tensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet 정규화
])
# S3 데이터셋 생성
bucket_name = 'smwu-cv-team-2-s3'
train_prefix = 'preprocessed-data/origin_split/train/'  # S3 폴더 경로
train_dataset = S3ImageDataset(bucket_name=bucket_name, prefix=train_prefix, transform=transform)
# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# 테스트 데이터 경로
test_prefix = 'preprocessed-data/origin_split/test/'
# S3에서 테스트 데이터셋 생성
test_dataset = S3ImageDataset(bucket_name=bucket_name, prefix=test_prefix, transform=transform)
# DataLoader 생성
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
6:49
val_prefix = 'preprocessed-data/origin_split/valid/'  # S3에 Validation 데이터 경로
val_dataset = S3ImageDataset(bucket_name=bucket_name, prefix=val_prefix, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)